# Gestion de al informacion en una BD

## Creación de Tablas

Acciones:

- CREATE
- ALTER
- DROP

### pgadmin

Para mostrar el proceso desde cero, eliminaremos por completo la BD *transporte*

    drop database transporte_publico;

Sobre la parte izquierda, click, create database, solo le damos el nombre, y dejamos como propietario al usuario *postgres*

![](https://i.imgur.com/0EYRSoU.png)

Fijate en la pestaña SQL, esta el comando que si quisieramos podriamos ejecutar en la consola.

![](https://i.imgur.com/C1XpH6d.png)

Le damos que si, y ya tenemos un BD vacia.

![](https://i.imgur.com/LuzGQho.png)

Toda esa configuracion como los triggers, extension, los veremos mas adelante. De manera predeterminada, se creo un esquema que es el publico. 

![](https://i.imgur.com/yIQ14hj.png)

Y donde dice *tables* crearemos nuestra primera tabla:

![](https://i.imgur.com/UHC8jGg.png)

- serial es autoincremental
- character varying seria el equivalente a varchar en otros motores. 

####  Llave Primaria

ir a la pestaña *constraints* en la subpestaña *Primary key*, y el nombre que le vamos a colocar va a ser una convencion: *el nombre de la tabla* seguido de *pkey* 

Y por ahora no modificaremos los parametros avanzados. Solo inspeccionaremos la pestaña SQL, comprobamos todo este bien, y la creamos(en nuestro caso modificamos):

![](https://i.imgur.com/yUaf0L1.png)

#### Script de insercion:

Desde pgadmin muy facil: INSERT script

![](https://i.imgur.com/YTTjszo.png)

Como id es un campo que se incrementa automaticamente con cada INSERT, una buena practica es quitarlo del Script de insercion.

Para ver el formato de inserccion de fechas, lo podemos verificar en el archivo de configuracion, o mejor aun, hacerle una consulta a la BD de la fecha actual

    SELECT current_date;

Y listo, lo tenemos:

    INSERT INTO public.pasajero(
	name, home_address, birthdate)
	VALUES ('Primer Pasajero', 'alguna direccion', '2023-04-07');

## PARTICIONES

![](https://i.imgur.com/rZBOppR.png)

En ocasiones llega haber demasiada informacion en una tabla, y para consultarla. se tendra que recorrer toda la informacion para encontrar lo que se esta buscando. 

Por ejemplo, la tabla viajes, va a tener alrededor de un millon de filas, y queremos saber los viajes entre tal fecha y tal otra, ¿Que se puede hacer?

Se podria hacer una separacion fisica de los datos, pero que conserve la misma estructura logica, es decir guardar diferentes partes de la tabla, en diferentes partes de disco, incluso en otros discos. Y como tiene la misma estructura logica, podriamos por ejemplo usar el mismo *SELECT*.

Y creamos, por ejemplo, una particion para enero, otra para febrero, etc, etc. Y si le damos una consulta de un mes en particular, no buscaria en toda la tabla, sino en la correspondiente a ese mes. 

### Practica

Crearemos una nueva tabla: *bitacora_viajes* con tres campos:

- id
- id_viaje
- fecha

Siendo fecha el campo por el cual vamos a particionar la tabla. 

Hacerlo desde *pgadmin*. 

![](https://i.imgur.com/rmLTnNs.png)

❌ Fijate no le colocamos primary key ni not null, para que cuando vayamos a la tab de constrains, no salga ninguna. 

👉 Importante aclarar que en tablas particionadas, no se usan llaves primarias porque no es posible

- En la pestaña general le indicamos que es una tabla particionada

Hasta ahora esto es lo que hemos hecho si revisamos la SQL tab, pero todavia no esta listo:

    CREATE TABLE public.bitacora_viajes
    (
        id serial,
        id_viaje integer,
        fecha date
    ) PARTITION BY RANGE (fecha);

    ALTER TABLE IF EXISTS public.bitacora_viajes
        OWNER to postgres;

Pero ojo todavia no se ha agregado una tabla particionanada, por tanto en este punto ni siquiera podremos insertar datos, ni nada. Igualmente la creamos. Y nota el icono es diferente:

![](https://i.imgur.com/I0LVaX8.png)

Si en este punto, intetaramos hacer una insercion, mira que el error que saldria:

    ERROR:  no partition of relation "bitacora_viajes" found for row
    DETAIL:  Partition key of the failing row contains (fecha) = (2010-05-05).
    SQL state: 23514

Entonces creemos una particion, hagamolo como una consulta SQL:

    CREATE TABLE bitacora_viajes201005 PARTITION OF bitacora_viajes
    FOR VALUES FROM ('2010-05-01') TO ('2010-05-31');

- La particion en realidad es una tabla, por lo que es importante crear tambien un nombre descriptivo: que es del mes de mayo.

Ahora si intentenomos nuevamente insertar una fila dentro de ese rango:

    INSERT INTO public.bitacora_viajes(
	id_viaje, fecha)
	VALUES (1, '2010-05-05');


### Ejercicio. 

Crear una particion para el año 2009, e insertar un dato:

    CREATE TABLE bitacora_viajes2009 PARTITION OF bitacora_viajes
    FOR VALUES FROM ('2009-01-01') TO ('2009-12-31');

Y el valor:

    INSERT INTO public.bitacora_viajes(
	    id_viaje, fecha)
	    VALUES (2, '2009-12-30');


Y hagamos un select general, para verificar hay dos datos en la tabla:

    SELECT * FROM bitacora_viajes;

Explorando un poco mas el arbol de la BD:

![](https://i.imgur.com/qWAwIqn.png)

No se usaran partciones en el proyecto. Asi que si quieres borrala. 